In [ ]:
# Se importan las librerias necesarias
import mne
import pandas as pd
import numpy as np
import os
from mne.time_frequency import psd_array_welch
import matplotlib.pyplot as plt 

In [45]:
def data_reading_txt(filename_eeg_txt):
    ''' Función encargada de leer los archivos de EEG en formato txt'''
    # Leemos txt con pandas
    raw_data = pd.read_csv(filename_eeg_txt, skiprows=4)
    # Obtenemos columnas de tiempo y triggers
    other = raw_data.iloc[:, 12:19].to_numpy()
    analog_channel = raw_data.iloc[:, 19:22].to_numpy()
    time = raw_data[' Timestamp'].to_numpy()
    time = time-time[0]
    # Obtenemos columnas de canales y convertimos a uV 
    raw_data_eeg = raw_data.iloc[:, 1:9].to_numpy()*2.235174e-8
    # Creamos el objeto info 
    ch_names = ['Ch1','Ch2','Ch3','Ch4','Ch5', 'Ch6', 'Ch7', 'Ch8'] 
    ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
    sfreq = 2000
    eeg_info = mne.create_info(ch_names, sfreq, ch_types)
    # Creamos el objeto raw
    eeg_data = mne.io.RawArray(np.transpose(raw_data_eeg), eeg_info)

    return eeg_data

In [46]:
# Ruta base donde están las carpetas "Con Musica" y "Sin Musica"
base_dir = r"C:\Users\lucas\OneDrive\Desktop\Facultad\TIF\Senales"

In [47]:
# Lista para almacenar resultados
resultados = []

In [ ]:
# Recorrer ambas condiciones
for condition in ["Con Musica", "Sin Musica"]:
    condition_path = os.path.join(base_dir, condition)

    # Recorrer todas las carpetas de personas dentro de cada condición
    for person_folder in os.listdir(condition_path):
        person_path = os.path.join(condition_path, person_folder)

        # Confirmar que sea una carpeta
        if os.path.isdir(person_path):
            eeg_file = os.path.join(person_path, "eeg.txt")
            csv_file = os.path.join(person_path, "sart.csv")

            # Verificar que el archivo exista
            if os.path.exists(eeg_file):
                
                # Leer los datos EEG
                raw = data_reading_txt(eeg_file)
                
                # Obtener los nombres actuales de los primeros 3 canales
                canales_antiguos = raw.ch_names[:3]

                # Definir los nuevos nombres según lo que ustedes registraron, si no son esos cambien el nombre
                canales_nuevos = ['Fz', 'Cz', 'Pz']

                # Crear un diccionario {antiguo: nuevo}
                rename_dict = dict(zip(canales_antiguos, canales_nuevos))

                # Renombrar
                raw.rename_channels(rename_dict)
                
                # Duracion de la senal
                duracion_total = raw.times[-1]  # en segundos
                
                df = pd.read_csv(csv_file)
                
                # Filtramos todas las filas donde thisTrialN sea 0 o 134
                df_filtered = df[df["thisN"].isin([0, 134])].copy()

                # Localizamos la segunda aparición de thisTrialN == 0  (Se supera la fase de prueba)
                idx_0 = df_filtered.index[df_filtered["thisN"] == 0]
                if len(idx_0) >= 2:
                    second_0_index = idx_0[1]  # segunda ocurrencia
                else:
                    raise ValueError("Solo se encontró una ocurrencia de thisN == 0")

                # Extraemos esa fila y la que tiene thisTrialN == 134
                row_second_0 = df_filtered.loc[second_0_index]
                row_134 = df_filtered[df_filtered["thisN"] == 134].iloc[0]

                # Unimos ambas en un nuevo DataFrame
                result = pd.DataFrame([row_second_0, row_134])

                # Tiempo donde se concluye toda la actividad
                if "trial_4.stopped" in df.columns:
                    trial4_value = df["trial_4.stopped"].dropna().iloc[0]  # toma el primero no vacío
                    
                # Extraer valores de tiempo
                t1, t2 = result["thisRow.t"].values

                # Calculo la duracion de la actividad
                delta_t = trial4_value - t1
                
                t_inicio = max(duracion_total - 154, 0)  # 154 segundos hacia atrás
                raw_ultimos_2min = raw.copy().crop(tmin=t_inicio, tmax=duracion_total)
                
                # Seleccionar los canales
                canales = ['Fz', 'Cz', 'Pz'] # Son los 3 canales que utilizan
                raw_sel = raw_ultimos_2min.copy().pick_channels(canales)
                
                # Obtener los datos como array (shape: n_channels x n_times)
                data = raw_sel.get_data()  # por defecto en Voltios
                sfreq = raw_sel.info['sfreq']

                # Parámetros para ventanas de 1 s con 50% de solapamiento
                n_fft = int(sfreq * 1.0)       # 1 s
                noverlap = int(sfreq * 0.5)    # 0.5 s

                # Calcular PSD
                psd, freqs = psd_array_welch(
                    data, sfreq=sfreq, fmin=4, fmax=30,
                    n_fft=n_fft, n_overlap=noverlap
                )
                
                theta_band = (4, 8)
                beta_band  = (13, 30)
                theta_idx = (freqs >= theta_band[0]) & (freqs <= theta_band[1])
                beta_idx  = (freqs >= beta_band[0])  & (freqs <= beta_band[1])
                
                theta_power = np.mean(psd[:, theta_idx], axis=1)  # promedio sobre frecuencias
                beta_power  = np.mean(psd[:, beta_idx], axis=1)
                
                theta_beta_ratio = theta_power / beta_power
                
                # Promedio sobre los 3 canales
                theta_power_mean = np.mean(theta_power)
                beta_power_mean  = np.mean(beta_power)

                # Razón theta/beta promedio
                theta_beta_ratio_mean = theta_power_mean / beta_power_mean

                # Localizamos todos los índices donde thisN == 0
                indices_0 = df.index[df["thisN"] == 0]

                if len(indices_0) >= 2:
                    # Índice de la segunda aparición de thisN == 0
                    inicio_segundo_bloque = indices_0[1]

                    # Nos quedamos con todas las filas desde ese índice hasta el final
                    df_segundo_bloque = df.loc[inicio_segundo_bloque:].copy()

                # Número de respuestas correctas en el segundo bloque (NaN se ignoran)
                num_correctas = df_segundo_bloque["respCorr"].sum()  # si son 1/0, NaN se ignoran

                # Promedio de tiempo de respuesta (NaN se ignoran automáticamente)
                if "key_resp.rt" in df.columns:
                    promedio_respuesta = df_segundo_bloque["key_resp.rt"].mean()
                else:
                    promedio_respuesta = np.nan
                
        resultados.append({
        "Participante": person_folder,
        "Condición": condition,
        "RTB": theta_beta_ratio_mean,
        "Numero de aciertos sobre 134 " : num_correctas,
        "Tiempo de respuesta promedio" : promedio_respuesta
        })

Creating RawArray with float64 data, n_channels=8, n_times=923668
    Range : 0 ... 923667 =      0.000 ...   461.834 secs
Ready.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=8, n_times=1294236
    Range : 0 ... 1294235 =      0.000 ...   647.117 secs
Ready.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=8, n_times=1379106
    Range : 0 ... 1379105 =      0.000 ...   689.553 secs
Ready.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=8, n_times=867281
    Range : 0 ... 867280 =      0.000 ...   433.640 secs
Ready.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Effective window size : 1.000 (s)
Creating RawArray wi

In [ ]:
# Se almacenan los resultados en formato excel
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("resultados.xlsx", index=False)